In [1]:
pwd

u'/home/houruijie/traffic/traffic/src'

In [13]:
#encoding:utf-8
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
plt.figure(figsize=(15,18))
import os
import sys
import re
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import xgboost as xgb
%matplotlib inline
predict_dir_path = '/home/houruijie/traffic/traffic/data/predict_data/'
train_dir_path = '/home/houruijie/traffic/traffic/data/train/'

In [3]:
def concat_goods_counts():
    #将读取的第一个CSV文件写入合并后的文件保存
    file_list = os.listdir(train_dir_path + '货量表/')
    df = pd.read_csv(train_dir_path+ '货量表/' + file_list[0]) 
    df.to_csv(train_dir_path + 'goods_counts.csv',index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv(train_dir_path + '货量表/' + file_list[i])
        df.to_csv(train_dir_path + 'goods_counts.csv',index=False, header=False, mode='a+')
concat_goods_counts()

In [4]:
def concat_call_logs():
    #将读取的第一个CSV文件写入合并后的文件保存
    file_list = os.listdir(train_dir_path + '司机电话日志/')
    df = pd.read_csv(train_dir_path+ '司机电话日志/' + file_list[0]) 
    df.to_csv(train_dir_path + 'call_logs.csv',index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv(train_dir_path + '司机电话日志/' + file_list[i])
        df.to_csv(train_dir_path + 'call_logs.csv',index=False, header=False, mode='a+')
driver_call_logs = pd.read_csv(train_dir_path + 'call_logs.csv' )

In [5]:
# 司机画像
driver_portrait = pd.read_csv(train_dir_path + '司机画像.csv')
# 货主画像
shipper_portrait = pd.read_csv(train_dir_path+'货主画像.csv')
# 城市公路距离
city_way_distance = pd.read_csv(train_dir_path + '城市公路距离记录.csv')
# 城市天气记录
city_weather_history = pd.read_csv(train_dir_path + '城市天气记录.csv', encoding='gbk')
city_weather_history.date = pd.to_datetime(city_weather_history['date'], format='%Y-%m-%d', errors='ignore')
# 货量表
goodscounts = pd.read_csv(train_dir_path + 'goods_counts.csv')
goodscounts.day = pd.to_datetime(goodscounts['day'], format='%Y%m%d', errors='ignore')
# 司机电话日志
driver_call_logs = pd.read_csv(train_dir_path + 'call_logs.csv' )
# 司机量表
driver_counts = pd.read_csv(train_dir_path + 'driver_counts.csv')
driver_counts.day = pd.to_datetime(driver_counts['day'], format='%Y%m%d', errors='ignore')


In [6]:
# merge weather data
goods_counts_add_weather = goodscounts.merge(city_weather_history, left_on=['city', 'day'], right_on=['code', 'date'],)
driver_counts_add_weather = driver_counts.merge(city_weather_history, left_on=['city', 'day'], right_on=['code', 'date'])
goods_counts_add_weather['is_train'] = 1
driver_counts_add_weather['is_train'] = 1

In [7]:
# 预测数据
# predict = pd.read_csv(predict_dir_path + 'predict.csv', header=None)
weather = pd.read_csv(predict_dir_path + '天气数据.csv', encoding='gbk')
weather.date = pd.to_datetime(weather['date'], format='%Y-%m-%d', errors='ignore')
# predict_sample = pd.read_csv(predict_dir_path + 'predict_sample.csv', header=None)
predict_data = pd.read_csv(predict_dir_path + 'sample.csv')
predict_data.day = pd.to_datetime(predict_data['day'], format='%Y%m%d', errors='ignore')
predict_data = predict_data.merge(weather, left_on=['city', 'day'], right_on=['code', 'date'])
predict_data['is_train'] = 0

In [8]:
goods_counts_features = ['city', 'day', 'weather', 'temperature','wind','is_train']
driver_counts_features = ['city', 'day', 'weather', 'temperature','wind','is_train']

In [9]:
def get_max_temperature(string):
    return int(string.split('/')[0])
def get_min_temperature(string):
    return int(string.split('/')[1])
def get_diff_temperature(string):
    return int(string.split('/')[0]) - int(string.split('/')[1])
# wind
def get_number_wind(string):
    pattern = re.compile(ur'\d')
    return max(map(int, list(pattern.findall(string))))
def feature_engineering(goods_counts_add_weather):
    # date
    goods_counts_add_weather['weekday'] = goods_counts_add_weather.day.dt.weekday
    goods_counts_add_weather['monthofday'] = goods_counts_add_weather.day.dt.day
    # temperature
    goods_counts_add_weather['max_temperature'] = goods_counts_add_weather.temperature.apply(get_max_temperature)
    goods_counts_add_weather['min_temperature'] = goods_counts_add_weather.temperature.apply(get_min_temperature)
    goods_counts_add_weather['diff_temperature'] = goods_counts_add_weather.temperature.apply(get_diff_temperature)
    # weather
    count_vect = CountVectorizer()
    weather = pd.DataFrame(count_vect.fit_transform(goods_counts_add_weather["weather"]).toarray(), columns=count_vect.get_feature_names())
    goods_counts_add_weather = pd.concat([goods_counts_add_weather, weather],axis=1)
    # wind
    goods_counts_add_weather['wind'] = goods_counts_add_weather.wind.apply((get_number_wind))
    # feature select
    no_use_featuer = ['day', 'name', 'code', 'date','weather', 'temperature']
    feature = [x for x in goods_counts_add_weather if x not in no_use_featuer]
    return goods_counts_add_weather[feature]

In [10]:
goods_counts_data = feature_engineering(pd.concat([goods_counts_add_weather[goods_counts_features], predict_data[goods_counts_features]], ignore_index=True))
goods_counts_train_data = goods_counts_data[goods_counts_data.is_train==1]
goods_counts_predict_data = goods_counts_data[goods_counts_data.is_train==0]

In [11]:
driver_counts_data = feature_engineering(pd.concat([driver_counts_add_weather[driver_counts_features], predict_data[driver_counts_features]], ignore_index=True))
driver_counts_train_data = driver_counts_data[driver_counts_data.is_train==1]
driver_counts_predict_data = driver_counts_data[driver_counts_data.is_train==0]

## goos_counts_prediction

In [12]:
no_use_feature = ['is_train']
features = [x for x in goods_counts_train_data.columns if x not in no_use_feature]
train_data = goods_counts_train_data[features]
test = goods_counts_predict_data[features]
label = goods_counts_add_weather['count']

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5
    

lgb_params1 = {}
lgb_params1['application'] = 'regression'
lgb_params1['boosting'] = 'gbdt'
lgb_params1['learning_rate'] = 0.015
lgb_params1['num_leaves'] = 32
lgb_params1['min_sum_hessian_in_leaf'] = 2e-2
lgb_params1['min_gain_to_split'] = 0
lgb_params1['bagging_fraction'] = 0.9
lgb_params1['feature_fraction'] = 0.9
lgb_params1['num_threads'] = 4
lgb_params1['metric'] = 'rmse'

lgb_params2 = {}
lgb_params2['application'] = 'regression'
lgb_params2['boosting'] = 'gbdt'
lgb_params2['learning_rate'] = 0.02
lgb_params2['lambda_l1'] = 0.5
lgb_params2['num_leaves'] = 32
lgb_params2['min_gain_to_split'] = 0
lgb_params2['bagging_fraction'] = 0.8
lgb_params2['feature_fraction'] = 0.8
lgb_params2['num_threads'] = 4
lgb_params2['metric'] = 'rmse'

lgb_params3 = {}
lgb_params3['application'] = 'regression'
lgb_params3['boosting'] = 'gbdt'
lgb_params3['learning_rate'] = 0.022
lgb_params3['num_leaves'] = 32
lgb_params2['lambda_l2'] = 0.3
lgb_params3['bagging_freq'] = 8
lgb_params3['min_gain_to_split'] = 0
lgb_params3['bagging_fraction'] = 0.8
lgb_params3['feature_fraction'] = 0.8
lgb_params3['num_threads'] = 4
lgb_params3['metric'] = 'rmse'

def do_train(X_train, X_valid, y_train, y_valid, lgb_params, rounds):
    X_t = X_train.values
    y_t = y_train.values
    d_train = lgb.Dataset(X_t, y_t)
    X_v = X_valid.values
    y_v = y_valid.values
    d_valid = lgb.Dataset(X_v, y_v)
    watchlist = [d_train, d_valid]
    lgb_model = lgb.train(lgb_params, train_set=d_train, num_boost_round=rounds, 
                          valid_sets=watchlist, verbose_eval=1000, early_stopping_rounds = 300)
    test_pred = lgb_model.predict(X_v)
    rmsle = RMSLE(y_v, test_pred)
#     print(X_t)
#     print(lgb_model.feature_importance())
    return rmsle, lgb_model

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=74)
rmsle, lgb_model1 = do_train(X_train, X_valid, y_train, y_valid, lgb_params1, 12000)
test_pred1 = lgb_model1.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)
    
X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=2121)
rmsle, lgb_model2 = do_train(X_train, X_valid, y_train, y_valid, lgb_params2, 10000)
test_pred2 = lgb_model2.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=4)
rmsle, lgb_model3 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred3 = lgb_model3.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=19)
rmsle, lgb_model4 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred4 = lgb_model4.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)  

#test_pred = (test_pred3 + test_pred4) / 2
test_pred = (test_pred1 + test_pred2 + test_pred3 + test_pred4) / 4
# result.to_csv('LGB_sub.csv', index=False)
print('Done')

Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 129.833	valid_1's rmse: 278.427
Early stopping, best iteration is:
[999]	training's rmse: 129.914	valid_1's rmse: 278.374
Test RMSLE: 278.374
Training until validation scores don't improve for 300 rounds.
Early stopping, best iteration is:
[671]	training's rmse: 143.493	valid_1's rmse: 293.003
Test RMSLE: 293.003
Training until validation scores don't improve for 300 rounds.
Early stopping, best iteration is:
[495]	training's rmse: 185.48	valid_1's rmse: 288.097
Test RMSLE: 288.097
Training until validation scores don't improve for 300 rounds.
Early stopping, best iteration is:
[552]	training's rmse: 171.693	valid_1's rmse: 301.741
Test RMSLE: 301.741
Done


In [29]:
submit = pd.read_csv(predict_dir_path + 'sample.csv')
submit['cargo_count'] = test_pred.astype(int)


## driver_call_counts

In [31]:
no_use_feature = ['is_train']
features = [x for x in driver_counts_train_data.columns if x not in no_use_feature]
train_data = driver_counts_train_data[features]
test = driver_counts_predict_data[features]
label = driver_counts_add_weather['driver_call_count']

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5

lgb_params1 = {}
lgb_params1['application'] = 'regression'
lgb_params1['boosting'] = 'gbdt'
lgb_params1['learning_rate'] = 0.015
lgb_params1['num_leaves'] = 32
lgb_params1['min_sum_hessian_in_leaf'] = 2e-2
lgb_params1['min_gain_to_split'] = 0
lgb_params1['bagging_fraction'] = 0.9
lgb_params1['feature_fraction'] = 0.9
lgb_params1['num_threads'] = 4
lgb_params1['metric'] = 'rmse'

lgb_params2 = {}
lgb_params2['application'] = 'regression'
lgb_params2['boosting'] = 'gbdt'
lgb_params2['learning_rate'] = 0.02
lgb_params2['lambda_l1'] = 0.5
lgb_params2['num_leaves'] = 32
lgb_params2['min_gain_to_split'] = 0
lgb_params2['bagging_fraction'] = 0.8
lgb_params2['feature_fraction'] = 0.8
lgb_params2['num_threads'] = 4
lgb_params2['metric'] = 'rmse'

lgb_params3 = {}
lgb_params3['application'] = 'regression'
lgb_params3['boosting'] = 'gbdt'
lgb_params3['learning_rate'] = 0.022
lgb_params3['num_leaves'] = 32
lgb_params2['lambda_l2'] = 0.3
lgb_params3['bagging_freq'] = 8
lgb_params3['min_gain_to_split'] = 0
lgb_params3['bagging_fraction'] = 0.8
lgb_params3['feature_fraction'] = 0.8
lgb_params3['num_threads'] = 4
lgb_params3['metric'] = 'rmse'

def do_train(X_train, X_valid, y_train, y_valid, lgb_params, rounds):
    X_t = X_train.values
    y_t = y_train.values
    d_train = lgb.Dataset(X_t, y_t)
    X_v = X_valid.values
    y_v = y_valid.values
    d_valid = lgb.Dataset(X_v, y_v)
    watchlist = [d_train, d_valid]
    lgb_model = lgb.train(lgb_params, train_set=d_train, num_boost_round=rounds, 
                          valid_sets=watchlist, verbose_eval=1000, early_stopping_rounds = 300)
    test_pred = lgb_model.predict(X_v)
    rmsle = RMSLE(y_v, test_pred)
#     print(X_t)
#     print(lgb_model.feature_importance())
    return rmsle, lgb_model

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=74)
rmsle, lgb_model1 = do_train(X_train, X_valid, y_train, y_valid, lgb_params1, 12000)
test_pred1 = lgb_model1.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)
    
X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=2121)
rmsle, lgb_model2 = do_train(X_train, X_valid, y_train, y_valid, lgb_params2, 10000)
test_pred2 = lgb_model2.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=4)
rmsle, lgb_model3 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred3 = lgb_model3.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=19)
rmsle, lgb_model4 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred4 = lgb_model4.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)  

#test_pred = (test_pred3 + test_pred4) / 2
test_pred = (test_pred1 + test_pred2 + test_pred3 + test_pred4) / 4
# result.to_csv('LGB_sub.csv', index=False)
print('Done')

Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 400.447	valid_1's rmse: 820.132
Early stopping, best iteration is:
[868]	training's rmse: 432.446	valid_1's rmse: 818.738
Test RMSLE: 818.738
Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 337.394	valid_1's rmse: 940.287
Early stopping, best iteration is:
[1072]	training's rmse: 322.271	valid_1's rmse: 939.306
Test RMSLE: 939.306
Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 389.496	valid_1's rmse: 894.816
Early stopping, best iteration is:
[782]	training's rmse: 445.876	valid_1's rmse: 891.434
Test RMSLE: 891.434
Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 353.635	valid_1's rmse: 1012.68
Early stopping, best iteration is:
[1146]	training's rmse: 322.632	valid_1's rmse: 1006.96
Test RMSLE: 1006.956
Done


In [34]:
submit['driver_call_count'] = test_pred.astype(int)

In [36]:
submit.to_csv('submit07101052.csv', header=False)

In [ ]:
model = xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=160, silent=False, objective='reg:gamma')
model.fit(train_data, label, verbose = True)
predict_data['cargo_count'] = model.predict(goods_counts_predict_data[features])

In [13]:

model = xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=300, silent=1, objective='reg:gamma')
model.fit(train_data, label, verbose = True)
predict_data['driver_call_count'] = model.predict(goods_counts_predict_data[features])

In [ ]:
no_use_feature = ['is_train']
features = [x for x in goods_counts_train_data.columns if x not in no_use_feature]
train_data = goods_counts_train_data[features]
test = goods_counts_predict_data[features]
label = goods_counts_add_weather['count']

In [2]:
submit = pd.read_csv(predict_dir_path + 'sample.csv')

In [4]:
submit['cargo_count'] = 100000

In [6]:
submit['driver_call_count'] = 100000

In [7]:
submit['cargo_count'] = predict_data.cargo_count.astype(int)
submit['driver_call_count'] = predict_data.driver_call_count.astype(int)


NameError: name 'predict_data' is not defined

In [8]:
submit.to_csv('submit07082300.csv', header=False)

In [9]:
submit

,city,day,cargo_count,driver_call_count
0,320100,20180601,100000,100000
1,320100,20180602,100000,100000
2,320100,20180603,100000,100000
3,320100,20180604,100000,100000
4,320100,20180605,100000,100000
5,320100,20180606,100000,100000
6,320100,20180607,100000,100000
7,320200,20180601,100000,100000
8,320200,20180602,100000,100000
9,320200,20180603,100000,100000


array([  605.16400146,   584.75445557,   562.63470459,   663.8637085 ,
         611.75286865,   611.75286865,   622.91265869,  2556.32348633,
        2478.07543945,  2152.61572266,  2547.78076172,  2398.51416016,
        2129.01855469,  2124.86230469,   784.7354126 ,   885.46447754,
         784.19415283,   860.190979  ,   843.5690918 ,   832.7522583 ,
         873.73535156,  1869.60424805,  1660.71936035,  1749.23986816,
        1938.00732422,  1767.02600098,  1767.02600098,  1770.96179199,
         868.4196167 ,   722.35223389,   693.99066162,   758.45849609,
         800.10650635,   711.37329102,   709.98455811,   717.03503418,
         523.94207764,   490.09213257,   571.17266846,   546.17498779,
         499.94116211,   498.96517944,   367.10897827,   310.2104187 ,
         287.15914917,   398.69961548,   367.55877686,   333.62039185,
         286.28659058,   548.27893066,   579.11340332,   572.56884766,
         589.61785889,   447.35205078,   454.82385254,   453.93597412,
      

In [69]:
df = pd.read_csv(predict_dir_path + 'predict.csv')
df

,96380588048193900,398829615
0,96436390618420100,399055125
1,96436391955588500,399018641
2,96052842979150500,399245056
3,96436391432418500,399215994
4,96436390683149300,398941636
5,96042790853897100,398806249
6,23429835901030500,398934006
7,96117370309881700,398739517
8,96380588049660100,398660048
9,95838243405212900,399044576


In [6]:
# time feature
goods_counts_add_weather['weekday'] = goods_counts_add_weather.day.dt.weekday
goods_counts_add_weather['monthofday'] = goods_counts_add_weather.day.dt.day

In [7]:
# temperature
def get_max_temperature(string):
    return int(string.split('/')[0])
def get_min_temperature(string):
    return int(string.split('/')[1])
def get_diff_temperature(string):
    return int(string.split('/')[0]) - int(string.split('/')[1])
goods_counts_add_weather['max_temperature'] = goods_counts_add_weather.temperature.apply(get_max_temperature)
goods_counts_add_weather['min_temperature'] = goods_counts_add_weather.temperature.apply(get_min_temperature)
goods_counts_add_weather['diff_temperature'] = goods_counts_add_weather.temperature.apply(get_diff_temperature)

In [8]:
# add weather feature
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
weather = pd.DataFrame(count_vect.fit_transform(goods_counts_add_weather["weather"]).toarray(), columns=count_vect.get_feature_names())
goods_counts_add_weather = pd.concat([goods_counts_add_weather, weather],axis=1)

In [9]:
# wind
import re
def get_number_wind(string):
    pattern = re.compile(ur'\d')
    return max(map(int, list(pattern.findall(string))))
goods_counts_add_weather['wind'] = goods_counts_add_weather.wind.apply((get_number_wind))

In [11]:
goods_counts_add_weather.shape

(1950, 42)

In [13]:
goods_counts_add_weather.columns

Index([            u'city',              u'day',            u'count',
                   u'name',             u'code',             u'date',
                u'weather',      u'temperature',             u'wind',
                u'weekday',       u'monthofday',  u'max_temperature',
        u'min_temperature', u'diff_temperature',               u'中雨',
                   u'中雨大雨',               u'中雪',             u'中雪大雪',
                     u'刮风',               u'多云',               u'大雨',
                   u'大雨暴雨',               u'大雪',             u'大雪暴雪',
                     u'小雨',             u'小雨中雨',               u'小雪',
                   u'小雪中雪',               u'少云',             u'局部多云',
                   u'晴间多云',               u'暴雨',               u'暴雪',
                     u'薄雾',               u'阴天',               u'阵雨',
                     u'阵雪',              u'雨夹雪',             u'零散阵雨',
                   u'零散雷雨',              u'雷阵雨',               u'雷雨'],
      dtype='object

In [17]:
no_use_featuer = ['day', 'name', 'code', 'date','weather', 'temperature', 'wind']
feature = [x for x in goods_counts_add_weather if x not in no_use_featuer]
goods_counts_add_weather[feature]

,city,count,weekday,monthofday,max_temperature,min_temperature,diff_temperature,中雨,中雨大雨,中雪,...,暴雪,薄雾,阴天,阵雨,阵雪,雨夹雪,零散阵雨,零散雷雨,雷阵雨,雷雨
0,320700,196,0,1,10,2,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,320700,411,1,2,10,2,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,320700,470,2,3,10,1,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,320700,345,3,4,3,0,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,320700,404,4,5,1,5,-4,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,320700,377,5,6,2,7,-5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,320700,268,6,7,4,2,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,320700,351,0,8,5,1,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,320700,371,1,9,4,6,-2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,320700,407,2,10,3,6,-3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
model = xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=160, silent=False, objective='reg:gamma')